In [253]:
import imaplib
import subprocess
import pandas as pd
from utils import *

imap_host = subprocess.check_output('op read op://Private/qggbe7eu3xeapptffcpq6aogku/server', shell=True).decode("utf-8").rstrip()
imap_user = subprocess.check_output('op read op://Private/qggbe7eu3xeapptffcpq6aogku/benutzername', shell=True).decode("utf-8").rstrip()
imap_pass = subprocess.check_output('op read op://Private/qggbe7eu3xeapptffcpq6aogku/passwort', shell=True).decode("utf-8").rstrip()

In [254]:
import pickle

# load pickle
cv = pickle.load(open("data/cv.pkl", "rb"))
model = pickle.load(open("data/model.pkl", "rb"))

In [255]:
# connect to host using SSL
imap = imaplib.IMAP4_SSL(imap_host)

## login to server
imap.login(imap_user, imap_pass)

imap.select('INBOX')

('OK', [b'17064'])

In [256]:
status, [data] = imap.search(None, '(ALL)')
#check
if status == "OK":
    messages = data.split()
    messages.reverse()
else:
    raise Exception

In [257]:
df = pd.DataFrame(columns=['msg_num','from','subject','body'])

for num in messages[0:500]:
	result, data = imap.fetch(num, '(RFC822)')
	print(f"Read message: {int(num)}")
	try:
		msg = email2Text(data[0][1])
	except (LookupError,TypeError):
		print(f"Skip message {int(num)}")
		continue
	df.loc[int(num)] = [num,msg['from'],msg['subject'],clean_up_pipeline(msg["body"][0])]
	#break


Read message: 17064
Read message: 17063
Read message: 17062
Read message: 17061
Read message: 17060
Read message: 17059
Read message: 17058
Read message: 17057
Read message: 17056
Read message: 17055
Read message: 17054
Read message: 17053
Read message: 17052
Read message: 17051
Read message: 17050
Read message: 17049
Read message: 17048
Read message: 17047
Read message: 17046
Read message: 17045
Read message: 17044
Read message: 17043
Read message: 17042
Read message: 17041
Read message: 17040
Read message: 17039
Read message: 17038
Read message: 17037
Read message: 17036
Read message: 17035
Read message: 17034
Read message: 17033
Read message: 17032
Read message: 17031
Read message: 17030
Read message: 17029
Read message: 17028
Read message: 17027
Read message: 17026
Read message: 17025
Read message: 17024
Read message: 17023
Read message: 17022
Read message: 17021
Read message: 17020
Read message: 17019
Read message: 17018
Read message: 17017
Read message: 17016
Read message: 17015


In [258]:
df.reset_index(drop=True,inplace=True)
df

,msg_num,from,subject,body
0,b'17064',Migros-Shop <news@reply.mail.migros-shop.de>,Neu: viele Produkte mit Mengenrabatt 🛒,kaffee schümli für € ab stück im browser anzei...
1,b'17063',Planet Sports <planetsports@sib1.planetsports....,Entdecke neue Ware & tolle Angebote!,der newsletter wird nicht richtig angezeigt hi...
2,b'17062',mail <mail@doerr-finanzen.de>,box Warning,dataimagepngbaseivborwkggoaaaansuheugaaagiaaaa...
3,b'17061',Google <no-reply@accounts.google.com>,Sicherheitswarnung für fwolpert@geospin.de,das ist eine kopie der an fwolpertgeospinde ge...
4,b'17060',BZ-Card-Reise-Newsletter <bzcard-service@bzcar...,"Herr Wolpert, Reiselust? Sichern Sie sich exkl...",falls sie diese email nicht oder nur teilweise...
...,...,...,...,...
495,b'16569',Airbnb <express@airbnb.com>,RE: Buchung der Unterkunft „The sky is a neigh...,opentrack re buchung der unterkunft „the sky i...
496,b'16568',Airbnb <express@airbnb.com>,RE: Buchung der Unterkunft „The sky is a neigh...,opentrack re buchung der unterkunft „the sky i...
497,b'16567',Mein Grundeinkommen <no-reply@mein-grundeinkom...,Mittwoch verlosen wir 25 Grundeinkommen! Bist ...,bestätige jetzt deine teilnahme und lade freun...
498,b'16566',Jessica Steinkamp <j.steinkamp@zorwentas.com>,Update: Diese Regeln gelten bis zum 01.04.2023,sehr geehrte damen und herren diesen herbst we...


In [259]:
features = cv.transform(df['body'])
pd.DataFrame(data=features.toarray(),columns=cv.get_feature_names_out())

,aacecefecalcdafbunptpcidemailreceiptxclick,aacecefeppidrtcnacderstadeded,aafcabcffefamptypetoguestwakeup,ab,abandoned,abbestell,abbuchung,abcheck,abdeck,abdomax,...,zweigniederlassung,zweisprachig,zwischensumm,zwischenzeitlich,zwnj,zwpghkjkyrehmoxneusnnrupxgjfhhrkksdfbgasypdzlosdcxy,zxsweogdnrmswhdphdlcvimgsaernxlkqaabujwcfmqomvvtscvzvwacmqdo,zxtipnbhcumxuypwfdwgxirgaikfsbxztrzebtvipcqzeoetzfnjfxzniviyybbooggsbdrgokscrumwa,zysvpvnkabfrbumduodefqgddcolugpkssefsjgtvtqeajcglwexgyuoeijbsctkntzfnxvyvhqsoinuklxtzivsdanjqxxhugktnzclzxavccutcbpq,å¾
0,0,0,0,13,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,3,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,3,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
496,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
497,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
498,0,0,0,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [260]:
cv.get_feature_names_out()

array(['aacecefecalcdafbunptpcidemailreceiptxclick',
       'aacecefeppidrtcnacderstadeded', 'aafcabcffefamptypetoguestwakeup',
       ...,
       'zxtipnbhcumxuypwfdwgxirgaikfsbxztrzebtvipcqzeoetzfnjfxzniviyybbooggsbdrgokscrumwa',
       'zysvpvnkabfrbumduodefqgddcolugpkssefsjgtvtqeajcglwexgyuoeijbsctkntzfnxvyvhqsoinuklxtzivsdanjqxxhugktnzclzxavccutcbpq',
       'å¾'], dtype=object)

In [261]:
predictions = model.predict(features)
predictions

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False,  True, False, False,
        True,  True, False, False, False, False, False, False, False,
       False,  True, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,

In [262]:
df['label'] = pd.Series(predictions)
df

,msg_num,from,subject,body,label
0,b'17064',Migros-Shop <news@reply.mail.migros-shop.de>,Neu: viele Produkte mit Mengenrabatt 🛒,kaffee schümli für € ab stück im browser anzei...,False
1,b'17063',Planet Sports <planetsports@sib1.planetsports....,Entdecke neue Ware & tolle Angebote!,der newsletter wird nicht richtig angezeigt hi...,False
2,b'17062',mail <mail@doerr-finanzen.de>,box Warning,dataimagepngbaseivborwkggoaaaansuheugaaagiaaaa...,False
3,b'17061',Google <no-reply@accounts.google.com>,Sicherheitswarnung für fwolpert@geospin.de,das ist eine kopie der an fwolpertgeospinde ge...,False
4,b'17060',BZ-Card-Reise-Newsletter <bzcard-service@bzcar...,"Herr Wolpert, Reiselust? Sichern Sie sich exkl...",falls sie diese email nicht oder nur teilweise...,False
...,...,...,...,...,...
495,b'16569',Airbnb <express@airbnb.com>,RE: Buchung der Unterkunft „The sky is a neigh...,opentrack re buchung der unterkunft „the sky i...,False
496,b'16568',Airbnb <express@airbnb.com>,RE: Buchung der Unterkunft „The sky is a neigh...,opentrack re buchung der unterkunft „the sky i...,False
497,b'16567',Mein Grundeinkommen <no-reply@mein-grundeinkom...,Mittwoch verlosen wir 25 Grundeinkommen! Bist ...,bestätige jetzt deine teilnahme und lade freun...,False
498,b'16566',Jessica Steinkamp <j.steinkamp@zorwentas.com>,Update: Diese Regeln gelten bis zum 01.04.2023,sehr geehrte damen und herren diesen herbst we...,False


## Select spam mails

In [263]:
df_spam = df[df['label']]
df_spam

,msg_num,from,subject,body,label
51,b'17013',Jens Rahnfeld <rahnfelj@informatik.uni-freibur...,AW: Continuous Integration HistomAIte,hi maria florian florian mega cool dass du dir...,True
54,b'17010',Maria Kalweit <maria.kalweit@intelligent-oncol...,Re: Continuous Integration HistomAIte,lieber florian ich finde deinen vorschlag supe...,True
55,b'17009',Bernardin Dörr <bernardin.doerr@mertelsmann-fo...,Re: Continuous Integration HistomAIte,hallo zusammen ich kann inhaltlich zu florians...,True
64,b'17000',Disney+ <disneyplus@mail.disneyplus.com>,Deine Disney+ Abo-ID,disney email zwnj zwnj zwnj zwnj zwnj zwnj zwn...,True
133,b'16931',DAV Sektion Freiburg-Breisgau <info@dav-freibu...,DAV Freiburg-Breisgau Newsletter Dezember 2022,neues aus deiner sektion webansicht webview ap...,True
177,b'16887',"""Amazon.de"" <versandbestaetigung@amazon.de>","Deine Amazon.de-Bestellung mit ""Cloer 1627-11 ...",amazonde versandbestätigung bestellnummer gute...,True
183,b'16881',Gregg <flowering@year.loverow.shop>,This new smartwatch_could save your life one day!,myrjuqvhdewdoprwopcwkobwfcfepbhozglpmehnlkujqz...,True
205,b'16859',"""Wikimedia Deutschland e. V."" <mitglieder@wiki...",Wikiversum: Ein Hoch auf das digitale Ehrenamt,aktuelles für mitglieder von wikimedia deutsch...,True
206,b'16858',Mein Grundeinkommen <no-reply@mein-grundeinkom...,"Wann, wenn nicht jetzt: Hoffnung schenken!",wie feiern wir weihnachten in der krise unsere...,True
219,b'16845',Georg Behringer <georg@behringer-finanzmakler.de>,VHV KFZ- Vertrag/ Vers.schein für neuen Skoda,hi flo hi christiane gute nachrichten für euch...,True


## Flag emails that appeared to be spam

In [264]:
df_spam.apply(lambda x: imap.store(x['msg_num'],'FLAGS','\\Flagged $MailFlagBit1'),axis=1)

51     (OK, [b'17013 (FLAGS (\\Flagged $MailFlagBit1)...
54     (OK, [b'17010 (FLAGS (\\Flagged $MailFlagBit1)...
55     (OK, [b'17009 (FLAGS (\\Flagged $MailFlagBit1)...
64     (OK, [b'17000 (FLAGS (\\Flagged $MailFlagBit1)...
133    (OK, [b'16931 (FLAGS (\\Flagged $MailFlagBit1)...
177    (OK, [b'16887 (FLAGS (\\Flagged $MailFlagBit1)...
183    (OK, [b'16881 (FLAGS (\\Flagged $MailFlagBit1)...
205    (OK, [b'16859 (FLAGS (\\Flagged $MailFlagBit1)...
206    (OK, [b'16858 (FLAGS (\\Flagged $MailFlagBit1)...
219    (OK, [b'16845 (FLAGS (\\Flagged $MailFlagBit1)...
268    (OK, [b'16796 (FLAGS (\\Flagged $MailFlagBit1)...
295    (OK, [b'16769 (FLAGS (\\Flagged $MailFlagBit1)...
300    (OK, [b'16764 (FLAGS (\\Flagged $MailFlagBit1)...
342    (OK, [b'16722 (FLAGS (\\Flagged $MailFlagBit1)...
372    (OK, [b'16692 (FLAGS (\\Flagged $MailFlagBit1)...
415    (OK, [b'16649 (FLAGS (\\Flagged $MailFlagBit1)...
439    (OK, [b'16625 (FLAGS (\\Flagged $MailFlagBit1)...
446    (OK, [b'16618 (FLAGS (\\

In [265]:
imap.close()

('OK', [b'Close completed (0.001 + 0.000 + 0.001 secs).'])